In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import HTML
import random
import math
import seaborn as sns

In [ ]:
## M/M/1 Simulation
trialsdict = {} # Dictionary containing all trials
trialgraphdatadict={}
t=0

while t <= 19:

#Storing variables for each customer in Lists
  Boxes=[]      # Box count
  IAT_=[]       # Inter-Arrival times
  SerTime=[]    # Service times
  ArrTime=[]    # Arrival times
  WaitTime=[]   # Wait times
  DepartTime=[] # Departure times
  SystemTime=[] # System times

  lambdaArr= 40/3600 # 40 Per Hour
  lambdaService= 50/3600 # 50 Per Hour


  #First Box
  ArrTime.append(0) #Arrival time for first Box = 0
  IAT_.append(0)    #IAT for first Box = 0
  WaitTime.append(0) # Wait time for first Box =0
  SerTime.append(np.random.exponential(1/lambdaService)) # Service Time --> Exponentially Distributed
  DepartTime.append(ArrTime[0]+WaitTime[0]+SerTime[0]) # Departure time = Arrival time + Wait time + Service time
  SystemTime.append(DepartTime[0]-ArrTime[0])          # System time = Departure time - Arrival time
  Boxes.append(1)

  # Print Wait times and System times
  #print ("For Box {} the Waiting Time is {}".format(1,WaitTime[0]))
  #print ("For Box {} the System Time is {}".format(1,SystemTime[0]))
 # print("")

  #Rest of Boxes
  for i in range (1,50):
      IAT_.append(np.random.exponential(1/lambdaArr))  # IAT ---> Exponentially Distributed
      ArrTime.append(ArrTime[i-1]+IAT_[i])           # Arrival Time= Previous Arrival time + IAT
      SerTime.append(np.random.exponential(1/lambdaService))   # Service Time --> Exponentially Distributed
      WaitTime.append(max(0,DepartTime[i-1]-ArrTime[i])) # Wait time = max (0, Departure time of pervious customer - Arrival time of current customer) , 0 if server is idle
      DepartTime.append(ArrTime[i]+WaitTime[i]+SerTime[i])   # Departure time = Arrival time + Wait time + Service time
      SystemTime.append(DepartTime[i]-ArrTime[i])            # System time = Departure time - Arrival time
      Boxes.append(i+1)

      # Print Wait times and System times
      #print ("For Box {} the Waiting Time is {}".format(i+1,WaitTime[i]))
      #print ("For Box {} the System Time is {}".format(i+1,SystemTime[i]))
      #print("")


  total_simulation_time_hours = DepartTime[-1] / 3600

  # Calculate the arrival rate (lambda) in boxes per hour
  arrival_rate = 50 / total_simulation_time_hours

  # Calculate the total service time in hours
  total_service_time_hours = sum(SerTime) / 3600

  # Calculate the service rate (mu) in boxes per hour
  service_rate = 50 / total_service_time_hours

  # Calculate Lq (mean number of boxes in the queue)
  mean_number_in_queue = (arrival_rate ** 2) / (service_rate * (service_rate - arrival_rate))

  # Calculate ρ (utilization)
  utilization = (arrival_rate / service_rate)



  t+=1
  print("Trial number {}".format(t))
  print("")

  # Print Average System time and Average Waiting time
  print("")
  print("Average System Time is {}".format(sum(SystemTime)/50))
  print("Average Waiting Time is {}".format(sum(WaitTime)/50))
  print("Arrival Rate is {} / hr".format(arrival_rate))
  print("Service Rate is {} / hr".format(service_rate))
  print("Mean number of boxes in queue is {}".format(mean_number_in_queue))
  print("Utilization Rate (ρ) is  {}".format(utilization))
  print("")

  #Creating a dictionary for each trial
  trial = {'Box' : Boxes,
        'IAT' : IAT_,
        'Arrival Time' : ArrTime,
        'Service Time': SerTime,
        'Wait Time' : WaitTime,
        'Departure Time' : DepartTime,
        'System Time' : SystemTime,
           }
  trialgraphdata = {'Utilization Rate': round(utilization,4),
                    'Mean_Number_of_Boxes': round(mean_number_in_queue,4),
                    'Mean Delay in System in HRS':round((sum(WaitTime)/50)/3600,4)}
  trialsdict[t]= trial # adding each trial dictionary to a dictionary that contains all trials
  trialgraphdatadict[t]=trialgraphdata

In [ ]:
#function to adjust the html in order to display multiple tables at once
def displayTable(*dfs):
    html = '<div style="display:flex , flex-direction:column">'
    for df in dfs:
        html += '<div style="margin-right: 2em">'
        html += df.to_html()
        html += '</div>'
    html += '</div>'
    display(HTML(html))


def round_two_digits(num):
  return round(num,2);



#iterating through the dictionary of trials and printing each one in a table
for i in range (1,21):
  print("")
  print("Trial number {}".format(i))
  print("")
  displayTable(pd.DataFrame(trialsdict[i]))



In [ ]:
trialtable_OG = pd.DataFrame(trialgraphdatadict)
trialtable = trialtable_OG.transpose()

sns.barplot(x = 'Utilization Rate',y = 'Mean_Number_of_Boxes',data = trialtable)
sns.set(rc={'figure.figsize':(11,8.27)})
plt.xticks(rotation=70)
plt.show()

In [ ]:
sns.barplot(x = 'Utilization Rate',y = 'Mean Delay in System in HRS',data = trialtable)
sns.set(rc={'figure.figsize':(11,8.27)})
plt.xticks(rotation=70)
plt.show()

In [ ]:
sns.barplot(x = 'Utilization Rate',y = 'Utilization Rate',data = trialtable)
sns.set(rc={'figure.figsize':(11,8.27)})
plt.xticks(rotation=70)
plt.show()

In [ ]:
# M/M/1-BBS Simulation

trialsdict = {}
trialgraphdatadict_BBS={}
# Initialize the Blum-Blum-Shub generator parameters
S0 = 31203056643
p = 4000000003
q = 8000000003
n = p * q  # The modulus

# Function to generate the next Blum-Blum-Shub random number
def BBS_generator(S):
    return (S**2) % n

# Function to generate an exponential random variable using BBS
def exp_rv_using_BBS(mean, Si):
    Si_plus_1 = BBS_generator(Si)
    ui_plus_1 = Si_plus_1 / (2 * n)
    return -mean * np.log(ui_plus_1), Si_plus_1

# Function to simulate the M/M/1-BBS queue with updated seed after each customer
def simulate_MM1_BBS_updated_seed(lambdaArr, lambdaService, num_customers=50, initial_seed=None):
    global trialsdict
    if initial_seed is None:
        initial_seed = S0
    Si = initial_seed
    Boxes=[]      # Box count
    IAT_ = [0]
    SerTime = []
    ArrTime = [0]
    WaitTime = [0]
    DepartTime = []
    SystemTime = []

    # First customer's service time
    exp_rv, Si = exp_rv_using_BBS(1/lambdaService, Si)
    SerTime.append(exp_rv)
    DepartTime.append(ArrTime[0] + WaitTime[0] + SerTime[0])
    SystemTime.append(DepartTime[0] - ArrTime[0])
    Boxes.append(1)

    # Process the rest of the customers
    for i in range(1, num_customers):
        exp_rv, Si = exp_rv_using_BBS(1/lambdaArr, Si)
        IAT_.append(exp_rv)
        ArrTime.append(ArrTime[-1] + IAT_[-1])

        exp_rv, Si = exp_rv_using_BBS(1/lambdaService, Si)
        SerTime.append(exp_rv)

        WaitTime.append(max(0, DepartTime[-1] - ArrTime[i]))
        DepartTime.append(ArrTime[i] + WaitTime[i] + SerTime[i])
        SystemTime.append(DepartTime[i] - ArrTime[i])
        Boxes.append(i+1)


    # Return the relevant simulation results
    return {
        'Box' : Boxes,
        'IAT':IAT_,
        'Arrival Times': ArrTime,
        'Service Times': SerTime,
        'Wait Times': WaitTime,
        'Departure Times': DepartTime,
        'System Times': SystemTime,
        'last_seed': Si  # Include the last seed for continuity in simulations
    }

# Function to run the simulation multiple times with updated seeds
def run_multiple_simulations(num_trials, lambdaArr, lambdaService, num_customers=50):
    trials_results={}
    current_seed = S0
    trial = 1
    accepted_trials = 0  # Counter for accepted trials

    while accepted_trials < num_trials:
        # Run the simulation with the updated seed
        results = simulate_MM1_BBS_updated_seed(lambdaArr, lambdaService, num_customers, current_seed)

        # Update the seed for the next trial
        current_seed = results['last_seed']

        # Calculate the total simulation time in hours
        total_simulation_time_hours = results['Departure Times'][-1] / 3600

        # Calculate the arrival rate (lambda) in boxes per hour
        arrival_rate = num_customers / total_simulation_time_hours

        # Calculate the total service time in hours
        total_service_time_hours = sum(results['Service Times']) / 3600

        # Calculate the service rate (mu) in boxes per hour
        service_rate = num_customers / total_service_time_hours

        # Calculate Lq (mean number of boxes in the queue)
        Lq = (arrival_rate ** 2) / (service_rate * (service_rate - arrival_rate)) if service_rate > arrival_rate else float('inf')

        # Calculate ρ (utilization)
        utilization = arrival_rate / service_rate


        #-- Calculate average system time and average waiting time
        average_system_time = np.mean(results['System Times'])
        average_waiting_time = np.mean(results['Wait Times'])

        # Store results
        trials_results[accepted_trials + 1] = {
            'total_simulation_time_hours': total_simulation_time_hours,
            'arrival_rate': arrival_rate,
            'service_rate': service_rate,
            'Lq': Lq,
            'utilization': utilization,
            'average_system_time': average_system_time,
            'average_waiting_time': average_waiting_time
        }

        trialgraphdata = {'Utilization Rate': round(utilization,4),
                'Mean_Number_of_Boxes': round(Lq,4),
                'Mean Delay in System in HRS':round(average_waiting_time/3600,4)}
        # Increment the count of accepted trials
        trialsdict[accepted_trials]=results
        trialgraphdatadict_BBS[accepted_trials]=trialgraphdata
        accepted_trials += 1

    return trials_results

# Parameters for the simulation
lambdaArr = 40/3600  # Arrival rate (40 per hour)
lambdaService = 50/3600  # Service rate (50 per hour)
num_trials = 20

# Run the simulation and calculate averages
accepted_trials_results_with_averages = run_multiple_simulations(num_trials, lambdaArr, lambdaService)

# Print out the detailed results of accepted trials
for trial_num, trial_results in accepted_trials_results_with_averages.items():
    print(f"Trial number {trial_num}")
    print(f"Average System Time: {trial_results['average_system_time']:.2f} seconds")
    print(f"Average Waiting Time: {trial_results['average_waiting_time']:.2f} seconds")
    print(f"Total Simulation Time (hours): {trial_results['total_simulation_time_hours']:.2f}")
    print(f"Arrival Rate (λ): {trial_results['arrival_rate']:.2f} boxes/hour")
    print(f"Service Rate (μ): {trial_results['service_rate']:.2f} boxes/hour")
    print(f"Mean Number in Queue (Lq): {trial_results['Lq']:.2f} boxes")
    print(f"Utilization (ρ): {trial_results['utilization']:.2f}")
    print("")



In [ ]:
#function to adjust the html in order to display multiple tables at once
def displayTable(*dfs):
    html = '<div style="display:flex , flex-direction:column">'
    for df in dfs:
        html += '<div style="margin-right: 2em">'
        html += df.to_html()
        html += '</div>'
    html += '</div>'
    display(HTML(html))


def round_two_digits(num):
  return round(num,2);



#iterating through the dictionary of trials and printing each one in a table
for i in range (0,20):
  print("")
  print("Trial number {}".format(i+1))
  print("")
  displayTable(pd.DataFrame(trialsdict[i]))


In [ ]:
trialtable_BBS_OG = pd.DataFrame(trialgraphdatadict_BBS)
trialtable_BBS = trialtable_BBS_OG.transpose()
sns.barplot(x = 'Utilization Rate',y = 'Mean_Number_of_Boxes',data = trialtable_BBS)
sns.set(rc={'figure.figsize':(11,8.27)})
plt.xticks(rotation=70)
plt.show()

In [ ]:
sns.barplot(x = 'Utilization Rate',y = 'Mean Delay in System in HRS',data = trialtable_BBS)
sns.set(rc={'figure.figsize':(11,8.27)})
plt.xticks(rotation=70)
plt.show()

In [ ]:
sns.barplot(x = 'Utilization Rate',y = 'Utilization Rate',data = trialtable_BBS)
sns.set(rc={'figure.figsize':(11,8.27)})
plt.xticks(rotation=70)
plt.show()

In [ ]:
comparisonTab ={
    '':["M/M/1","M/M/1-BBS"],
    'Average Mean Number of Boxes for 20 Trials':[trialtable['Mean_Number_of_Boxes'].sum()/20 , trialtable_BBS['Mean_Number_of_Boxes'].sum()/20],
    'Average Delay in HRS for 20 trials ':[trialtable['Mean Delay in System in HRS'].sum()/20,trialtable_BBS['Mean Delay in System in HRS'].sum()/20],
    'Average Utilization for 20 trials ':[trialtable['Utilization Rate'].sum()/20,trialtable_BBS['Utilization Rate'].sum()/20]
}
comparisonTab = pd.DataFrame(comparisonTab).set_index('')
comparisonTab.rename(index=str).index
comparisonTab = comparisonTab.transpose()

comparisonTab.rename({0: "M/M/1", 1: "M/M/1-BBS"} , axis = 'columns')
displayTable(comparisonTab)